In [ ]:
import numpy as np 
from PIL import Image
import matplotlib.pyplot as plt
import os
import random
import cv2
from glob import glob
from tqdm import tqdm

def create_directory(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [ ]:
#breast
image_list=glob('../../data/NIPA/breast/images/*.jpeg')
mask_list=[f.replace('images', 'masks') for f in image_list]
mask_list=[f.replace('.jpeg', '.npy') for f in mask_list]
save_path='../../data/NIPA/integrated_512/breast/'

class_list={
    "Background":0,
    "NT_stroma": 1,
    "NT_epithelial":2,
    "NT_immune": 3,
    "Tumor": 4,
    "TP_invasive": 5,
    "TP_in_situ": 6,
}
integrated_class={
    "Background":[],
    "Stroma": ["Background","NT_stroma", "NT_immune"],
    "Non_Tumor": ["NT_epithelial"],
    "Tumor": ["Tumor", "TP_invasive", "TP_in_situ"],
}
integrated_class_color={
    "Background":[255,0,0],
    "Stroma": [0,255,0],
    "Non_Tumor": [0,0,255],
    "Tumor": [255,255,0],
}

for i in tqdm(range(len(image_list))):
    image=np.array(Image.open(image_list[i]).resize((512,512)))
    mask=np.load(mask_list[i])
    mask=cv2.resize(mask, (512,512), interpolation=cv2.INTER_NEAREST)
    dst_mask=np.zeros((image.shape[0],image.shape[1],len(integrated_class)), dtype=np.uint8)
    for j, (class_name, sub_class_names) in enumerate(integrated_class.items()):
        for sub_class_name in sub_class_names:
            sub_class_idx=class_list[sub_class_name]
            dst_mask[:,:,j]+=np.where(mask[:,:,sub_class_idx], 1, 0).astype(np.uint8)
    background_preprocessing_mask=np.where(np.std(image, axis=2)>10,255,0).astype(np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    background_preprocessing_mask=cv2.morphologyEx(background_preprocessing_mask, cv2.MORPH_CLOSE, kernel)
    background_preprocessing_mask=np.where(background_preprocessing_mask==0,1,0).astype(np.uint8) 
    dst_mask[np.where(background_preprocessing_mask==1)]=0
    dst_mask[:,:,0]+=background_preprocessing_mask
    create_directory(os.path.join(save_path, 'images'))
    create_directory(os.path.join(save_path, 'masks'))
    np.save(os.path.join(save_path, 'masks', os.path.basename(mask_list[i])), dst_mask)
    Image.fromarray(image).save(os.path.join(save_path, 'images', os.path.basename(image_list[i])))


In [ ]:

plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.imshow(image)
plt.subplot(1,3,2)
plt.imshow(np.std(image, axis=2))
plt.subplot(1,3,3)
plt.imshow(background_preprocessing_mask, cmap='gray')